In [1]:
#run this to install the package
!pip install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from eep153_tools.sheets import read_sheets

## **From Sheet to DataFrame to Regression Object**

In [3]:
US_Data = "1OWluIYUGkmR6iFgSzJJa2ikEqqMT6BHbsKZyip236HU"

In [4]:
x = read_sheets(US_Data,sheet='Food Expenditures 2021')
x["j"] = x["food_name"]
x = x.set_index(['i','t','m','j'])["grams"].squeeze()

# Household characteristics
d = read_sheets(US_Data,sheet="Household Characteristics 2021")
d.columns.name = 'k'
d["sex"] = d["sex"].map({"Female": 1, "Male": 0})
d['log_household'] = np.log(d['HH_size'])
d.drop("HH_size", axis= 1, inplace=True)

# Fill blanks with zeros
d = d.replace(np.nan,0)

# Expenditures x may have duplicate columns
x = x.T.groupby(['i','t','m','j']).sum()
x = x.replace(0,np.nan) # Replace zeros with missing

# Take logs of expenditures; call this y
y = np.log(x)
d.set_index(['i','t','m'],inplace=True)

### Filter Data Down

In [5]:
# Step 1: Filter goods by frequency (top 400)
food_counts = y.groupby('j').count()
top_foods = food_counts.sort_values(ascending=False).head(350).index

# Filter x and y to only include these popular goods.
x_filtered = x[x.index.get_level_values('j').isin(top_foods)]
y_filtered = y[y.index.get_level_values('j').isin(top_foods)]

# Step 2: Instead of random sampling, use all households present in y_filtered.
# Get all unique household ids from y_filtered.
common_ids = y_filtered.index.get_level_values('i').unique()

# Filter y and d to keep only households in common_ids.
# (This is effectively taking the full data that has corresponding expenditures.)
y_final = y_filtered[y_filtered.index.get_level_values('i').isin(common_ids)]
d_final = d[d.index.get_level_values('i').isin(common_ids)]

# Filter y and d to match
# sample_ids = y_filtered.index.get_level_values('i').unique().to_series().sample(1750, random_state=42)
# y_sample = y_filtered[y_filtered.index.get_level_values('i').isin(sample_ids)]
# d_sample = d[d.index.get_level_values('i').isin(sample_ids)]

# Final checks
print("Shape of y_final:", y_final.shape)
print("Shape of d_final:", d_final.shape)

Shape of y_final: (85881,)
Shape of d_final: (6711, 3)


In [6]:
y_final

i       t     m    j                                                        
130378  2021  USA  Caesar dressing                                              1.589235
                   Cheese, parmesan, dry grated                                 0.732368
                   Chicken, ns as to part and cooking method, skin not eaten    5.136386
                   Coffee, brewed                                               6.396930
                   Coffee, instant, reconstituted                               5.703782
                                                                                  ...   
142310  2021  USA  Mayonnaise, regular                                          2.639057
                   Potato chips, plain                                          2.995732
                   Sugar substitute, sucralose, powder                          0.000000
                   Water, bottled, plain                                        6.201523
                   Wine, red     

## **Build Price Data**

In [7]:
#Add price data
p = pd.read_csv("proj3_min_cost_data - prices.csv")

# Change column names to actual unit values
p["u"] = "Kg"

p["m"] = "USA"

# Adjust price to per Kg from per g
p["price"] = p["price"] * 10

# Filter for only 2017/2018 prices
p = p[p["t"] == "2017/2018"]

# Change price data from 2017/2018 to 2021 using avg US CPIs in 2017/2018 and 2021
cpi_2021 = 271.0
cpi_2018 = 250.0
p["price"] = p["price"] * (cpi_2021/cpi_2018)

p["t"] = "2021"


codes = pd.read_csv("proj3_min_cost_data - recipes.csv")

codes = codes[["parent_foodcode", "parent_desc"]]
c = codes.groupby("parent_foodcode").first()
c.rename(columns = {"parent_foodcode": "j"})
p = p.join(c, on="j")
p_with_j = p # use later on for nutrition data
p = p.drop("j", axis=1)
p = p.rename(columns = {"parent_desc":"j"})

# Now prices
p = p.set_index(['t','m','j','u'])

# Compute medians of prices for particular time, place and unit
p = p.groupby(['t','m','j','u']).median()

# Just keep metric units
p = p.xs('Kg',level="u").squeeze().unstack('j')
# Identify the common keys between x_filtered.index.levels[-1] and p.columns
common_keys = p.columns.intersection(x_filtered.index.levels[-1])

# Now subset p with these keys and transpose the result.
p = p[common_keys].T

## **Run Regression**

In [8]:
from cfe import Regression

# Run the regression
result = Regression(y=y_final, d=d_final, compute_se=False, rectify=False, 
                    min_obs=10, min_prop_items=0.002)

In [ ]:
predicted = result.predicted_expenditures()
predicted.head()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Create DataFrame of actual vs predicted log expenditures
df = pd.DataFrame({
    'y': y_final,
    'yhat': result.get_predicted_log_expenditures()
})

# Drop missing values (some may not match if data was trimmed)
df = df.dropna()

# Plot
df.plot.scatter(x='yhat', y='y', alpha=0.4, figsize=(6, 6))
plt.title("Predicted vs Actual Log Expenditures")
plt.xlabel("Predicted log expenditure (ŷ)")
plt.ylabel("Actual log expenditure (y)")
plt.grid(True)
plt.axis("equal")  # force square axes
plt.show()

In [ ]:
ax = result.graph_beta()

In [ ]:
result.get_beta().sort_values(ascending=False)

In [ ]:
foods_in_regression = result.beta.index.get_level_values('j').unique()

In [ ]:
result.get_gamma().head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

gamma = result.get_gamma()
gamma_sorted = gamma.sort_values(by="Constant", ascending=False)
plt.figure(figsize=(10, 8))
sns.barplot(data=gamma_sorted, x="Constant", y="j")
plt.xlabel("Gamma (Constant)")
plt.ylabel("Food Item")
plt.title("Gamma Constants by Food Item")
plt.tight_layout()
plt.show()

In [ ]:
ax = result.get_w().plot.hist(bins=100,density=True)
result.get_w().plot.kde(ax=ax)

In [ ]:
ax = result.get_w()[(result.get_w() > -10) & (result.get_w() < 10)].plot.hist(bins=100,density=True)
result.get_w()[(result.get_w() > -10) & (result.get_w() < 10)].plot.kde(ax=ax)

In [ ]:
gamma = result.get_gamma()
gamma

## Saving Regression

In [ ]:
result.to_pickle('us_estimates_2021.rgsn')

In [ ]:
import cfe

# Load the result back into memory
result = cfe.regression.read_pickle('us_estimates_2021.rgsn')

## **Demand and Utility**

Having estimated the demand system, we can examine the implied demand curves.

## Budgets
Get food budget for all households, then find median budget:

In [ ]:
import numpy as np

xhat = result.predicted_expenditures()

# Total food expenditures per household
xbar = xhat.groupby(['i','t','m']).sum()

# Reference budget
xref = xbar.quantile(0.5)  # Household at 0.5 quantile is median

**Reference Prices**
  
Choose reference prices. Here we’ll choose a particular year, and average prices across markets. If you wanted to focus on particular market you’d do this differently.

In [ ]:
# Reference prices chosen from a particular time; average across place.
# These are prices per kilogram:
pbar = p.mean(axis=1)

# Instead of directly indexing with result.beta.index (which fails for missing keys),
# reindex pbar to have the same index as result.beta and fill missing prices with 1.
pbar = pbar.reindex(result.beta.index, fill_value=1)

# (Optional) Get the common keys, which should be all keys in result.beta.index:
common_keys = pbar.index.intersection(result.beta.index)
pbar = pbar.loc[common_keys]

# Replace any remaining missing values with 1.
pbar = pbar.replace(np.nan, 1)

# Finally, define a function to change the price of a single good in the price vector.
def my_prices(p0, j, p=pbar):
    """
    Change price of the jth good to p0, holding all other prices fixed.
    
    Parameters:
      p0 : float
          New price for the jth good.
      j : label
          Identifier for the good (must be present in p).
      p : pd.Series
          Price vector (default is pbar).
    
    Returns:
      pd.Series : A new price vector with the updated value.
    """
    p = p.copy()
    p.loc[j] = p0
    return p

common_keys

## **Demand Curve for One Food**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import warnings
%matplotlib inline

def plot_demand_curve(good):
    """
    Plot the demand curve for a given good over a range of price scales.
    
    Parameters
    ----------
    good : str
        The common key (food identifier) for which you want the demand curve,
        e.g. "Banana, raw".
        
    Global Variables (should be defined before calling this function):
      pbar         : pd.Series of reference prices (indexed by food identifier)
      result       : an object with a demands() method and attribute beta.index.
      xref         : reference expenditure vector (e.g., median household expenditures)
      xbar         : household total expenditures used to compute quantiles.
      my_prices()  : function that returns a modified price vector.
    """
    # Suppress warnings about negative beta values
    warnings.filterwarnings("ignore", message="Setting negative values of beta to zero")
    
    # Reindex pbar so that it has the same index as result.beta.index, filling missing keys with 1.
    pbar_local = pbar.reindex(result.beta.index, fill_value=1)
    
    # Define a range of scale factors: vary prices from 50% to 200% of the reference price.
    scale = np.linspace(0.5, 2, 20)
    
    # Compute demand for the given good at different price levels.
    # Wrap the list comprehensions in a warnings.catch_warnings block to suppress any warnings during computation.
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="Setting negative values of beta to zero")
        median_demand = [result.demands(xref, my_prices(pbar_local[good] * s, good, pbar_local))[good]
                         for s in scale]
        demand_q25 = [result.demands(xbar.quantile(0.25), my_prices(pbar_local[good] * s, good, pbar_local))[good]
                      for s in scale]
        demand_q75 = [result.demands(xbar.quantile(0.75), my_prices(pbar_local[good] * s, good, pbar_local))[good]
                      for s in scale]
    
    # Create the plot.
    plt.figure(figsize=(8, 6))
    plt.plot(median_demand, scale, marker="o", label="Median Budget")
    plt.plot(demand_q25, scale, marker="s", label="25th Percentile Budget")
    plt.plot(demand_q75, scale, marker="^", label="75th Percentile Budget")
    
    # Add axis labels, title, legend, and grid.
    plt.xlabel(f"Quantities of {good} Demanded")
    plt.ylabel(f"Price (relative to base of {pbar_local[good]:.2f})")
    plt.title(f"Demand Curve for {good}")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_demand_curve('Banana, raw')

## **Demand Curve for More than One good**

In [ ]:
def plot_demand_curves(goods):
    """
    Plot demand curves for multiple goods over a range of price scales.
    
    Parameters
    ----------
    goods : tuple or list of str
        The food identifiers (common keys) for which you want to plot demand curves.
        For example: ('Banana, raw', 'Apple, raw', 'Milk, whole')
    
    Global variables required:
      - pbar: pd.Series with reference prices (indexed by food identifiers)
      - result: an object with methods demands() and attribute beta.index
      - xref: reference expenditure vector (e.g., median expenditures)
      - xbar: household total expenditures (used for quantile calculations)
      - my_prices(): function to adjust the price vector
    """
    # Suppress specific warnings (from consumerdemands._utils)
    warnings.filterwarnings("ignore", message="Setting negative values of beta to zero")
    
    # Ensure pbar covers all keys in result.beta.index; fill missing values with 1.
    pbar_local = pbar.reindex(result.beta.index, fill_value=1)
    
    # Define a range of scale factors to vary the price from 50% to 200% of reference.
    scale = np.linspace(0.5, 2, 20)
    
    # Create a new figure for the plot.
    plt.figure(figsize=(10, 8))
    
    # Loop over each good in the input tuple/list.
    for good in goods:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="Setting negative values of beta to zero")
            # Compute the demand for the given good at various price levels.
            median_demand = [result.demands(xref, my_prices(pbar_local[good] * s, good, pbar_local))[good]
                             for s in scale]
            demand_q25 = [result.demands(xbar.quantile(0.25), my_prices(pbar_local[good] * s, good, pbar_local))[good]
                          for s in scale]
            demand_q75 = [result.demands(xbar.quantile(0.75), my_prices(pbar_local[good] * s, good, pbar_local))[good]
                          for s in scale]
        
        # Plot each demand curve with distinct line styles and markers.
        plt.plot(median_demand, scale,
                 label=f"{good} (Median Budget)")
        plt.plot(demand_q25, scale,
                 label=f"{good} (25th Percentile Budget)")
        plt.plot(demand_q75, scale,
                 label=f"{good} (75th Percentile Budget)")
    
    # Add labels, title, legend, and grid.
    plt.xlabel("Quantity Demanded")
    # This label uses the base reference price of the first good in the list for display.
    plt.ylabel(f"Price (relative to base of {pbar_local[goods[0]]:.2f})")
    plt.title("Demand Curves for Selected Goods")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Example usage:
plot_demand_curves(('Banana, raw', 'Milk, whole'))

## **Engle Curves**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define scale, say we vary the budget from 50% to 200% of xref
scale = np.linspace(0.5, 2.0, 20)

# Now plot
fig, ax = plt.subplots()
ax.plot(np.log(scale*xref),
        [result.expenditures(s*xref, pbar)/(s*xref) for s in scale])
ax.set_xlabel(f'log budget (relative to base of {xref:.0f})')
ax.set_ylabel('Expenditure share')
ax.set_title('Engel Curves')
plt.show()

In [ ]:
def plot_engel_curves(interesting_goods, xref, scale, pbar, result):
    """
    Plot Engel curves for a list of specified goods.
    
    Parameters:
      interesting_goods: list of str
          A list of goods (by food name or id) for which to plot the curve.
      xref: float
          The base household budget.
      scale: array-like
          A list or array of scaling factors for the budget (e.g., np.linspace(0.5,2,20)).
      pbar: pandas.Series or DataFrame
          The reference prices for foods. The index should contain the goods' names.
      result:
          An object with a method `expenditures(budget, pbar)` that returns a pandas Series 
          with expenditures for each food (keys corresponding to food names).
    
    Returns:
      fig, ax : matplotlib Figure and Axes objects.
    """
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Loop over each requested good and calculate its expenditure share at each budget.
    for good in interesting_goods:
        shares = []
        for s in scale:
            budget = s * xref  # Scale the base budget by s.
            exp_series = result.expenditures(budget, pbar)
            # Check if the good exists in the returned series
            if good in exp_series.index:
                share = exp_series[good] / budget
            else:
                # Print a warning if the good is not found and set the share to NaN.
                print(f"Warning: '{good}' not found in expenditure series at budget {budget:.2f}")
                share = np.nan
            shares.append(share)
            
        # Plot the Engel curve for this good using log(s * xref) on x-axis.
        ax.plot(np.log(scale * xref), shares, label=good)
    
    # Set axis labels and title.
    ax.set_xlabel(f'Log Budget (relative to base of {xref:.0f})', fontsize=12)
    ax.set_ylabel('Expenditure Share', fontsize=12)
    ax.set_title("Engel Curves for Selected Goods", fontsize=14, fontweight='bold')
    ax.legend(fontsize=8, loc='best')
    plt.tight_layout()
    
    return fig, ax

In [ ]:
# Example usage:
# Define your variables appropriately
xref = 3272  # example base budget
scale = np.linspace(0.5, 2, 20)  # example scale factors
# pbar and result must be defined elsewhere in your code:
# For example, pbar could be a pandas Series keyed by food names,
# and result should be an object with a method expenditures(budget, pbar).

interesting_goods = ["Avocado, raw", "Banana, raw", "Water, tap", "Apple, raw", "Onions, raw"]

fig, ax = plot_engel_curves(interesting_goods, xref, scale, pbar, result)
plt.show()

## **Indirect Utility**

In [ ]:
fig,ax = plt.subplots()

ax.plot(scale*xref,[result.indirect_utility(s*xref,pbar) for s in scale])
ax.set_xlabel(f'Indirect Utility (Budget relative to base of {xref:.0f}')
ax.set_ylabel(f'Utility')
ax.set_title('Indirect Utility Function')

# **[A] Nutritional Data**
### Step 1: Acquire DataFrames

In [ ]:
xhat.unstack('j')

In [ ]:
p

In [ ]:
p_new = pd.read_csv("proj3_min_cost_data - prices.csv")
p_new["u"] = "Kg"

p_new["m"] = "USA"

p_new["price"] = p_new["price"] * 10

p_new = p_new[p_new["t"] == "2017/2018"]


# Change price data from 2017/2018 to 2021 using avg US CPIs in 2017/2018 and 2021
cpi_2021 = 271.0
cpi_2018 = 250.0
p_new["price"] = p_new["price"] * (cpi_2021/cpi_2018)

p_new["t"] = "2021"

p_new = p_new.join(c, on="j") 
p_new = p_new.drop("j", axis=1)
p_new = p_new.drop("t", axis=1)
p_new = p_new.drop("u", axis=1)
p_new = p_new.rename(columns = {"parent_desc":"j"})

p_new

In [ ]:
p_pivot = p_new.pivot_table(
    index=["m"],   # or ["m", "t"] if you prefer that order
    columns="j",
    values="price",
    aggfunc="sum"      # or another aggregation if you have duplicates
)

p_pivot

In [ ]:
# Expenditures divided by prices/kg gives quantities in kgs...
xhat_unstacked = xhat.unstack('j')
xhat_unstacked_aligned = xhat_unstacked.reindex(index=p_pivot.index, columns=p_pivot.columns)
common_columns = xhat_unstacked_aligned.columns.intersection(p_pivot.columns)
common_index = xhat_unstacked_aligned.index.intersection(p_pivot.index)

p_pivot_aligned = p_pivot.loc[common_index, common_columns]

qhat = (xhat_unstacked/p_pivot_aligned).dropna(how='all')

# Drop missing columns
qhat = qhat.loc[:,qhat.count()>0]

In [ ]:
nutrients = pd.read_csv("proj3_min_cost_data - nutrients.csv")
recipes = pd.read_csv("proj3_min_cost_data - recipes.csv")

# from fndds diet problem: normalize weights to percentage terms. 
recipes['ingred_wt'] = recipes['ingred_wt']/recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")

# we're going to extend the recipes data frame to include the nutrient profiles of its ingredients (in 100g)
df = recipes.merge(nutrients, how="left", on="ingred_code")

# multiply all nutrients per 100g of an ingredient by the weight of that ingredient in a recipe.
numeric_cols = list(df.select_dtypes(include=["number"]).columns)
numeric_cols.remove("ingred_wt")
df[numeric_cols] = df[numeric_cols].mul(df["ingred_wt"], axis=0)
df = df.rename(columns={'parent_desc': 'j'})

# sum nutrients of food codes (over the multiple ingredients)
# python tip: one can merge dictionaries dict1 dict2 using **, that is: dict_merge = {**dict1, **dict2}. 
#The ** effectively "unpacks" the key value pairs in each dictionary
df = df.groupby('j').agg({**{col: "sum" for col in numeric_cols},
                                        "j": "first"})
df.drop("parent_foodcode", axis = 1, inplace=True)
df.drop("ingred_code", axis = 1, inplace=True)
df.drop("j", axis = 1, inplace=True)

df.index.name = "j"

fct = df

In [ ]:
fct.index

In [ ]:
qhat.columns

## **Map predicted quantities into nutrients**

In [ ]:
use = fct.index.intersection(qhat.columns)


nutrients = qhat[use]@fct.loc[use,:]
nutrients.mean()    # NB: Nutrients are for past /week/ for entire household. (Depends on dataset)

In [ ]:
rdi = pd.read_csv("proj3_min_cost_data - rda_new.csv")
rdi["n"] = rdi["Nutrient"]
rdi.drop("Nutrient", axis = 1, inplace=True)
rdi.drop("Nutrient Type", axis = 1, inplace=True)
rdi.drop("Unit", axis = 1, inplace=True)
rdi.drop("Constraint Type", axis = 1, inplace=True)
rdi = rdi.set_index("n")

In [ ]:
d = read_sheets(US_Data,sheet="Household Characteristics 2021")
d.columns.name = 'k'
d = d.replace(np.nan,0)

In [ ]:
# Define age bins and labels
bins = [0, 4, 9, 14, 19, 31, 51, 100]
labels = ['00-03', '04-08', '09-13', '14-18', '19-30', '31-50', '51-99']

# Assign age group based on bins
d['age_group'] = pd.cut(d['age'], bins=bins, labels=labels, right=False)

# Create column name like "Females 00-03", "Males 14-18", etc.
d['group'] = d['sex'] + 's ' + d['age_group'].astype(str)

# Count each person as 1
d['count'] = 1

# Pivot the table to wide format: rows = (i, t, m), columns = group
df_grouped = d.pivot_table(index=['i', 't', 'm'], columns='group', values='count',  aggfunc='sum', fill_value=0)

# Flatten the column index
df_grouped.columns.name = None
df_grouped = df_grouped.reset_index()

# Add log household size
df_hsize = d.drop_duplicates(['i', 't', 'm'])[['i', 't', 'm', 'HH_size']]
df_hsize['log HSize'] = np.log(df_hsize['HH_size'])

# Merge back
final_d = pd.merge(df_grouped, df_hsize.drop(columns='HH_size'), on=['i', 't', 'm'])

final_d.set_index(['i','t','m'],inplace=True)
final_d.head()

In [ ]:
final_d = final_d[rdi.columns.tolist()]

hh_rdi = final_d@rdi.T
hh_rwi = hh_rdi*7
hh_rdi

In [ ]:
# Match up nutrient names
use_nutrients = nutrients.columns.intersection(hh_rwi.columns)

nutrient_ratio = (nutrients[use_nutrients]/hh_rwi[use_nutrients]).dropna()

nutrient_ratio

In [ ]:
def plot_nutrient_histograms(nutrient_df, nutrients, log_transform=True, offset=1, bins=50, figsize=(8, 4)):
    """
    Plot histograms for one or more nutrients from a DataFrame.
    
    Parameters:
      nutrient_df : pandas.DataFrame
          DataFrame containing nutrient measurements in separate columns.
      nutrients : str or list
          The column name(s) for the nutrient(s) to plot.
      log_transform : bool (default True)
          If True, plot the log(values + offset) so that the distribution is on a log scale.
      offset : numeric (default 1)
          Added to the raw value before taking the logarithm (to avoid log(0)).
      bins : int (default 50)
          Number of histogram bins.
      figsize : tuple (default (10, 6))
          Figure size; if multiple nutrients are provided, the function creates one plot per nutrient vertically.
          
    Returns:
      fig : matplotlib.figure.Figure
          The figure object containing the histogram(s).
      axes : matplotlib.axes.Axes or array of Axes
          The Axes object(s) for further customization.
    """
    # If a single nutrient is passed as a string, convert to list.
    if isinstance(nutrients, str):
        nutrients = [nutrients]
    
    num_nutrients = len(nutrients)
    
    # Create a figure with one subplot per nutrient.
    fig, axes = plt.subplots(num_nutrients, 1, figsize=(figsize[0], figsize[1]*num_nutrients))
    
    # If only one nutrient, axes is a single Axes object, so wrap it in a list.
    if num_nutrients == 1:
        axes = [axes]
    
    # Loop over each nutrient and create its histogram.
    for ax, nutrient in zip(axes, nutrients):
        # Get the raw values and drop missing values.
        values = nutrient_df[nutrient].dropna().values
        
        # Optionally transform the values using a log, with an offset to avoid issues at 0.
        if log_transform:
            plot_values = np.log(values + offset)
            xlabel = f'log({nutrient} + {offset})'
        else:
            plot_values = values
            xlabel = nutrient
            
        # Calculate the mean of the transformed values.
        mean_val = plot_values.mean()
        
        # Plot the histogram with black bin borders.
        n, bin_edges, patches = ax.hist(plot_values, bins=bins, edgecolor='black', alpha=0.7, label=f'Distribution of {nutrient}')
        
        # Draw a vertical dashed red line at the mean.
        ax.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.2f}')
        
        # Label the axes and add a title for this subplot.
        ax.set_xlabel(xlabel, fontsize=12)
        ax.set_ylabel("Frequency", fontsize=12)
        ax.set_title(f"Histogram of {nutrient}", fontsize=14, fontweight='bold')
        ax.legend(fontsize=10, loc='best')
    
    plt.tight_layout()
    return fig, axes

In [ ]:
plot_nutrient_histograms(nutrient_ratio, ['Vitamin C', 'Vitamin D'])

In [64]:
print(nutrient_ratio['Vitamin E'].describe())

count    6692.000000
mean        2.778606
std        63.826337
min         0.019050
25%         1.131265
50%         1.576772
75%         2.230004
max      5215.829030
Name: Vitamin E, dtype: float64
